## Regularization and Optimization - Supervised Learning II - MDS Computational Linguistics

### Goal of this tutorial
- Implement regularization techniques (L2, L1, Dropout, Early stopping)
- Implement training optimization procedures (RMSProp, Adam)
- Revisit hyperparameter tuning
- Implement BiLSTM

### General
- This notebook was last tested on Python 3.8, PyTorch 1.7.1 and TorchText 0.8.1 (**Strongly recommended to use the same version for doing this tutorial and lab4**)


## Getting Started

In [7]:
# all the necessary imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch import optim
import torch
import torchtext
from torchtext.data import Field, LabelField
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

For this tutorial we will be using the HappyDB dataset (also used in lab3) to showcase how you can tune your neural networks.

## Part 1: Regularization

``Regularization`` refers to techniques you can apply to machine learning algorithms to counteract the impact of ``overfitting``. One way to do this is by giving a penalty to model parameters. There are a number of different ways to do this but we'll mainly be looking at the two main types of regularization, *L1* and *L2* regularization, as well as a technique called *dropout* which is a neural-network specific style of regularization.

## Model Capacity

An important concept you need to be aware of is that of ``model capacity``. While there are theoretical definitions of what model capacity is, you can generally think of it as the number of hypotheses (hypothesis space) a model can select. In deep learning, we can increase model capacity by increasing for example (1) ``depth``, by adding more hidden layers and (2) ``width``, by adding more nodes to the network hidden layers. A higher capacity model can fit very complex functions (find complicated solutions). Note: ``*Higher capacity models are more prone to overfitting, while lower capacity models can underfit*``.

### L2 Regularization

The most popular regularization technique in machine learning, L2 regularization, puts some penalty on model features based on the squared weights of these features, calculated using the L2-norm:  $\frac{\lambda}{2}||w||_2^2=\frac{\lambda}{2}\sum_i |w_i|^2$. The amount of regularization we apply to our model is a *hyperparameter* value indicated by $ \lambda$, some positive value (e.g., $0.001$) that will need to be optmized.

The goal of using a squared penalty on the value of the weights is to discourage models that have very large values for any particular weights, this in turn means that it's less likely to overfit to the particular patterns (features that has such highest features) it has seen in the training data.

The only major downside occurs in situations where your data has lots of *extraneous* features, in the case where you may have lots of garbage features mixed in with your data (much greater than the number of features that are useful for your problem) it may still try to use these features, leading to suboptimal results.

### Using L2 Regularization in Pytorch

In Pytorch L2 regularization is implemented through the inclusion *weight decay*, the name Pytorch uses to refer to ``L2 penalty``. Typically, when you set the value of ``weight_decay`` to be $>0$, L2 regularization will be used by the model. The following is an example of setting the value of ``weight_decay``: [Note we haven't declared a model yet, we'll run these things together at the end]

In [4]:
optimizer = torch.optim.SGD(model.parameters(), weight_decay = 0.001, lr=0.01)
#So easy!

### When to use L2 Regularization

Generally speaking L2 regularization is going to almost always help reduce overfitting in your problems. L2 regularization is also a popular method especially outside deep learning. For deep learning, ``dropout`` (introduced later) is currently a more popular approach to regularization and it does work well. You can also combine both L2 regularization and ``dropout``, although you don't usually need to.

The one area where L2 regularization might not quite be good enough, is as mentioned above, when you have really noisy data that includes a lot of irrelevant features. Instead, you might want to consider.

### L1 Regularization

*L1 regularization* functions similarly to L2 regularization, except it gives a linear penalty to the model weights using the L1 norm:  $\lambda ||w||_1 = \lambda \sum_i |w_i|$. The immediate upside of L1 regularization is that it can ignore extraneous features. By giving equal emphasis to all weight values, it will spend as much time trying to decrease the size of small weights as big weights, to the extent that it can shrink weight values to 0! Essentially this allows your model to perform automatic feature selection, getting rid of features that don't help (at least as much) solve the problem and leaving you with sparse weight matrices.

As a wonderful side effect of having the sparse matrices, this can (1) spead up training (due to less computations) and (2) decrease memory needed for storing your model. In addition, (3) L1-selected features (i.e., those not set to ``zero`` are ones that you can view as *relevant* and hence you can use them for ``interpretability`` (i.e., as a way to interpret why your model makes the decision it makes). This is important as a way to *see through* our models and help explain them, including to the general public. Increasingly, this is becoming more important over time as AI and its applications become more pervasive in our lives. 

### L1 Regularization in Pytorch

While Pytorch implements L1 Loss (an alternative to MSE loss that is used to deal with outliers in your dataset), it does not implement L1 regularization. That said it's somewhat straightforward to plug it into your models:

In [ ]:
#Here l1 is the total of the l1 norm of your paramters, and lmbd is your value for the l1 lambda (some positive value).
l1 = torch.tensor(0.)
for name, param in model.named_parameters():
    if 'weight' in name:
        l1 += torch.norm(param, p=1)
total_loss += cur_loss.item() + lmbd*l1

#### AKA

For your info: the L1 and L2 regularization techniques also have some other names they are refered to, sometimes people call L1 regularization ``LASSO`` and L2: ``Ridge`` regularization.


### Dropout

For a neural network specific regularization approach, dropout is a way to spread out what each node is learning. It does this by "dropping" some of the nodes from being used during an iteration of gradient descent, without those nodes, the rest of the nodes will then have to pick up the slack, thereby spreading out which nodes are used to understand different aspects of a problem. Dropout also has some really interesting properties when it comes to training, we'll look at this later in the tutorial. 

Ultimately, using dropout means that you are actually training several networks, rather then a single network, and somewhat using an ensemble of these networks to predict. This is particulary useful since it `forces` your model to distribute its dependence to more nodes by preventing ``co-adaptation`` between only particular nodes. In spite of its simplicity, ``dropout`` is elegant and effective (as stated earlier).

### Dropout in Pytorch

For ``LSTMs`` and ``RNNs`` dropout is dead simple [see LSTM documentation](https://pytorch.org/docs/stable/nn.html?highlight=lstm#torch.nn.LSTM):


In [ ]:
#just add "dropout=" and the value you want to apply [the probability of dropping any given unit (0,1)]
self.rnn_lstm = nn.LSTM(input_size=embedding_size, dropout=.7, hidden_size=hidden_size, num_layers=num_layers)
self.rnn_layer = nn.RNN(input_size=embedding_size, dropout=.7, hidden_size=hidden_size, num_layers=num_layers)

For ``feedforward`` neural networks add a dropout layer and then call this before the linear layer in your forward function ([see documentation](https://pytorch.org/docs/stable/nn.html?highlight=dropout#torch.nn.Dropout)):

In [ ]:
### inside model definition:
def __init__(self, num_inputs, hidden_layers, num_outputs):
    self.droplayer = nn.Dropout(p=.5)
    self.ll = nn.Linear(input_dim, hidden_layer)

def forward(self,x):
    #### somwhere in your forward function...
    out = self.droplayer(out)
    out = self.ll(out)

### Early stopping

[Early stopping](https://en.wikipedia.org/wiki/Early_stopping) is a simple regularization technique for models that use gradient descent for optimization. In this course so far, we have trained neural models for a fixed number of epochs (say 30). What if the performance of the model doesn't improve after some epochs? There is a chance that the model has likely started overfitting on the dataset. To prevent overfitting, early stopping approach uses some constraints to stop the training once the model starts overfitting. Some constraints can be:

- stop the training once performance on the validation set (say accuracy for sentiment classification task) obtained in the current epoch is poorer than the performance on the validation set obtained in the previous epoch (strict constraint)


In [ ]:
# start the training (CAN'T RUN THIS CELL)
prev_val_acc = None
for epoch in range(30):
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc, train_f1 = evaluate(train_iter)
    # compute the validation accuracy
    val_acc, val_f1 = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, 30, train_loss, train_acc, val_acc))

    # early stopping logic
    if prev_val_acc and prev_val_acc > val_acc:
        # val acc at current epoch is poorer compared to val acc at previous epoch 
        print('Training stopping early at epoch = %d'%epoch)
        break
    prev_val_acc = val_acc

- stop the training once performance on the validation set (say accuracy for sentiment classification task) doesn't improve for a fixed number (called as patience, a hyperparameter) of consecutive epochs (relaxed constraint)

In [ ]:
# start the training (CAN'T RUN THIS CELL)
prev_val_acc = None
cur_patience = 0
max_patience = 5 # hyperparameter
for epoch in range(30):
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc, train_f1 = evaluate(train_iter)
    # compute the validation accuracy
    val_acc, val_f1 = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, 30, train_loss, train_acc, val_acc))

    # early stopping logic
    if prev_val_acc and prev_val_acc > val_acc:
        # val acc at current epoch is poorer compared to val acc at previous epoch 
        cur_patience += 1
        if cur_patience == max_patience:
            # validation accuracy didn't improve for consecutive "max_patience" epochs 
            print('Training stopping early at epoch = %d'%epoch)
            break
    else:
        cur_patience = 0
    prev_val_acc = val_acc
    

### Final Words on Regularization

All of these approaches can be combined in interesting ways, you for instance could do L1, L2, early stopping and Dropout all at once, will it work better than other approaches? Only way to find out is through trying and then tuning the hyperparamenters.

Another thing to also consider is that while their main purpose is to improve generalizability of ML models, they can have interesting other effects on your network, L1 regularization improve the training time because of their properties of creating sparse matrices. Dropout does a similar thing. Don't underlook the value of these techniques in terms of saving time in the training/validating/testing process.

## Part 2 Optimization

Alongside using regularization, there are a number of different ways to improve the training process of your network. In ``Assignment 2``, we already looked at the impact of changing your ``learning rate`` on the performace of neural networks. This still is important, but there are some other methods you can use to improve (e.g., speed up) your gradient descent technique.

### Momentum

The classic approach to improving your training procedure is to introduce ``momentum`` into your SGD procedure. All this means is that a certain percentage of forward momentum from one iteration of SGD will be carried over to the next iteration. Why would you want to do this? Sometimes you can get stuck in local minimum before finding a global minimum, momentum will push you out of these local minimums and help your network converge on optimal configurations faster. For more information, you can check Section 8.3.2 [https://www.deeplearningbook.org/contents/optimization.html](https://www.deeplearningbook.org/contents/optimization.html).

#### Momentum in Pytorch

Dead simple, add "momentum=" into your SGD function [SGD documentation](https://pytorch.org/docs/stable/optim.html?highlight=sgd#torch.optim.SGD):

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=.9)

Common values for Momentum include .99, .9, .5, but try things out yourself!

### RMSProp

``Root Mean Squared Propogation`` is another common technique. It basically changes the gradient accumulation into an exponentially weighted moving average. We won't get into the details but you can check Section 8.5.2 [https://www.deeplearningbook.org/contents/optimization.html](here) for more info.

#### RMSProp in Pytorch
Just switch out your SGD optimizer for RMSprop. RMSprop needs to set an **alpha** value which indicates how much smoothing you want to apply between different iteration of the algorithm. And similar to SGD you can add momentum and weight decay if you want. [Documenation on RMSProp](https://pytorch.org/docs/stable/optim.html?highlight=rmsprop#torch.optim.RMSprop)

In [ ]:
optimizer = torch.optim.RMSprop(model.parameters(), lr=LEARNING_RATE, alpha=0.99)

### Adam

Adam is somewhat a combination of ``RMSprop`` and ``momentum`` (with some distinctions, see Section 8.5.3 [https://www.deeplearningbook.org/contents/optimization.html](here)). As such, it is a fast optimizer.

### Adam in Pytorch
Similarly to RMSProp, it has parameters that you can change to impact its performance, Adam has two of these parameters, called the **beta** values, one to impact the averaging it uses for the gradient, and the other the square of the gradient (respectively). [Adam documentation](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(),betas=(0.9, 0.999),lr=LEARNING_RATE)

### Last Thoughts on Optimization

Generally, although other optimizers have more recently been proposed, ``Adam`` and ``SGD + momentum`` are currently probably the two most popular optimizers. You can get more intuitions about each optimizer from this nice blog: https://ruder.io/optimizing-gradient-descent/  For the list of optimizers supported by PyTorch, visit: https://pytorch.org/docs/stable/optim.html

## Part 3 Hyperparameter Tuning

We've got a lot of variables to play with, and finding the best combination is a matter of experimentation (we already touched upon hyperparameter tuning in the team assignment for lab1). Generally you will want to use a validation set, or even use something like cross-validation, to experiment with different configurations of hyper parameters and then decide on the best values.

Hyperparameters can influence each other greatly. If you could test these out independently, things would just be a matter of finding the best learning rate, the best weight-decay, what optimizer etc, and then piecing them all together. This sadly isn't the case. You'll need to cover the grid of all possible combinations to see which values are going to work best. Usually an approach of picking a set of values over an interval and then running your models over and over again is the best way to find your best model. This is called a grid search, and generally ML papers will report what ranges they tested when reporting their results. 

For now, you could simply choose a set of 3 reasonable values of each of these most important hyperparameters and try these. You can find what a reasonable value is in research papers, or textbooks (such as the links from the Deep Learning book by Goodfellow et al. (2016) above). Over time, you will gain experience as to what to try.

### Hyperparameter Tuning with an LSTM

Back to the HappyDB example, let's test out these different configurations and see how they do. At your table pick which parameters you want to try out and run the training code with them. I've included my example run outputs at the bottom to give you a sense of what to expect.

#### Parameters:
- ``Model features`` (in our case just words)

#### Hyperparameters:
- ``Learning Rate``
- ``Optimizer`` (Type and values of Optimizer)
- ``Regularization`` (Type and values of Regularizer)
- ``Batch Size``
- ``Epochs`` (Number of epochs)

#### BiLSTM Tangent

In the model below I've also shown how to turn your uni-directional LSTM into a bi-directional LSTM (reading the sentence forward AND backwards). This is useful for certain types of tasks, such as Named Entity Recognition, and generally might learn a little faster. One note on changing from uni to BiLSTM, is that BiLSTM output dimensions will be double the output dimensions from the uni-directional LSTM (you literally run it twice, once forward and once backwards and then concatenate  the two results together to get passed on). Make sure whatever is receiving the output from the BiLSTM is set to double the dimensions for their input!

Let's do the seed initialization, the data preparation work:

In [2]:
# set the seed
manual_seed = 123
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_gpu = torch.cuda.device_count()
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

# tokenizer
def whitespace_tokenize(text):
    return text.strip().split()

# data loading using torchtext
TEXT = Field(sequential=True, tokenize=whitespace_tokenize, lower=False)
LABEL = Field(sequential=False, unk_token = None)
train, val, test = TabularDataset.splits(
               path="./data/happy_db/", # the root directory where the data lies
               train='train.tsv', validation="dev.tsv", test="test.tsv", # file names
               format='tsv',
               skip_header=True, # if your tsv file has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=[('tweet', TEXT), ('label', LABEL)])
TEXT.build_vocab(train, max_size=5000)
LABEL.build_vocab(train)
train_iter, val_iter, test_iter = BucketIterator.splits(
 (train, val, test), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(32,32,32),
 sort_key=lambda x: len(x.tweet), 
 sort=True,
# A key to use for sorting examples in order to batch together examples with similar lengths and minimize padding. 
 sort_within_batch=True
)

cpu


/anaconda3/envs/tony/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/anaconda3/envs/tony/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/anaconda3/envs/tony/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes f

Let's build the LSTM model for sociality classification:

In [18]:
class LSTMmodel(nn.Module):
  
  def __init__(self, embedding_size, vocab_size, output_size, hidden_size, num_layers):
    # In the constructor we define the layers for our model (same as our previous RNN)
    super(LSTMmodel, self).__init__()
    # word embedding lookup table
    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_size)
    self.embedding.weight.data.normal_(0.0,0.05) # mean=0.0, mu=0.05

    
    # core LSTM module
    #UPDATE DROPOUT HERE

    #baseline:
    #self.lstm_rnn = nn.LSTM(input_size=embedding_size, hidden_size=hidden_size, num_layers=num_layers) # input_size, hidden_size, num_layers

    #with dropout:
    self.lstm_rnn = nn.LSTM(input_size=embedding_size, dropout=.7, hidden_size=hidden_size, num_layers=num_layers) # input_size, hidden_size, num_layers

    #BiLSTM tangent: For the HW you might also try a bidirectional LSTM architecture, this is good for certain tasks such as
    #Named entity recognition. Just add bidirectional=true to your LSTM parameters list. NOTE THIS DOUBLES THE OUTPUT DIMENSION!
    #self.lstm_rnn = nn.LSTM(input_size=embedding_size, bidirectional=True, dropout=.7, hidden_size=hidden_size, num_layers=num_layers) # input_size, hidden_size, num_layers

    
    self.activation_fn = nn.Sigmoid()
    self.linear_layer = nn.Linear(hidden_size, output_size) 

    #For BiLSTM you end up doubling the output from the LSTM (concatenating the two directions together)
    #You'll need to double the input dimension into the linear layer to compensate!
    #self.linear_layer = nn.Linear(hidden_size*2, output_size) 


    self.softmax_layer = nn.LogSoftmax(dim=1)
  
  def forward(self, x):
    # In the forward function we define the forward propagation logic
    out = self.embedding(x)
    out, (h_state, c_state) = self.lstm_rnn(out) # h_0 initialized to zeros by default
    # classify based on the hidden representation at the last token
    out = out[-1] # unsqueeze converts 1D input (D dimension) into 2D input (1xD) 
    out = self.linear_layer(out)
    out = self.softmax_layer(out) # accepts 2D or more dimensional inputs
    return out

Let's set the values for the hyperparameters and create the model, the loss function.

In [19]:
EMBEDDING_SIZE = 300 
VOCAB_SIZE = 5002
NUM_CLASSES = 2
HIDDEN_SIZE = 500
NUM_LAYERS = 2
model = LSTMmodel(EMBEDDING_SIZE, VOCAB_SIZE, NUM_CLASSES, HIDDEN_SIZE, NUM_LAYERS)
model = model.to(device)
print(model)

LEARNING_RATE = 0.1
criterion = nn.NLLLoss()

LSTMmodel(
  (embedding): Embedding(5002, 300)
  (lstm_rnn): LSTM(300, 500, num_layers=2, dropout=0.7)
  (activation_fn): Sigmoid()
  (linear_layer): Linear(in_features=500, out_features=2, bias=True)
  (softmax_layer): LogSoftmax(dim=1)
)


Let's code up the train and evaluation logic:

In [20]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def train(loader):
    total_loss = 0.0
    # iterate throught the data loader
    num_sample = 0
    for batch in loader:
        # load the current batch
        batch_input = batch.tweet
        batch_output = batch.label
        
        batch_input = batch_input.to(device)
        batch_output = batch_output.to(device)
        # forward propagation
        # pass the data through the model
        model_outputs = model(batch_input)
        # compute the loss
        
        cur_loss = criterion(model_outputs, batch_output)
        
        #l1_pen = torch.tensor(0.)
        #lambda_1 = 0.00001
        #for name, param in model.named_parameters():
        #    if 'weight' in name:
        #        l1_pen += torch.norm(param, p=1)
        
        total_loss += cur_loss.item() #+ len(batch)*lambda_1*l1_pen

        # backward propagation (compute the gradients and update the model)
        # clear the buffer
        optimizer.zero_grad()
        # compute the gradients
        cur_loss.backward()
        # update the weights
        optimizer.step()

        num_sample += batch_output.shape[0]
    return total_loss/num_sample

# evaluation logic based on classification accuracy
def evaluate(loader):
    all_pred=[]
    all_label = []
    with torch.no_grad(): # impacts the autograd engine and deactivate it. reduces memory usage and speeds up computation
        for batch in loader:
             # load the current batch
            batch_input = batch.tweet
            batch_output = batch.label

            batch_input = batch_input.to(device)
            # forward propagation
            # pass the data through the model
            model_outputs = model(batch_input)
            # identify the predicted class for each example in the batch
            probabilities, predicted = torch.max(model_outputs.cpu().data, 1)
            # put all the true labels and predictions to two lists
            all_pred.extend(predicted)
            all_label.extend(batch_output)
            
    accuracy = accuracy_score(all_label, all_pred)
    f1score = f1_score(all_label, all_pred, average='macro') 
    return accuracy,f1score

Let's kick-start the training:

In [23]:
import warnings
warnings.filterwarnings('ignore')

manual_seed = 123
torch.manual_seed(manual_seed)
if n_gpu > 0:
    torch.cuda.manual_seed(manual_seed)

#Play around with the optimizer, L2 regularization, and momentum here.
#optimizer = torch.optim.RMSprop(model.parameters(),lr=LEARNING_RATE, alpha=0.99)
optimizer = torch.optim.Adam(model.parameters()) #,lr=LEARNING_RATE, betas=(0.9,0.99))
#optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=.9)
#optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
#optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=.9)
#optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, weight_decay=.00001)

# start the training
for epoch in range(10):
    # train the model for one pass over the data
    train_loss = train(train_iter)  
    # compute the training accuracy
    train_acc,f1t = evaluate(train_iter)
    # compute the validation accuracy
    val_acc,f1v = evaluate(val_iter)
    
    # print the loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Training Accuracy: {:.4f}, Validation Accuracy: {:.4f}'.format(epoch+1, 10, train_loss, train_acc, val_acc))
    

Epoch [1/10], Loss: 0.0009, Training Accuracy: 0.9812, Validation Accuracy: 0.8551
Epoch [2/10], Loss: 0.0008, Training Accuracy: 0.9766, Validation Accuracy: 0.8447
Epoch [3/10], Loss: 0.0005, Training Accuracy: 0.9961, Validation Accuracy: 0.8731
Epoch [4/10], Loss: 0.0003, Training Accuracy: 0.9943, Validation Accuracy: 0.8731
Epoch [5/10], Loss: 0.0003, Training Accuracy: 0.9962, Validation Accuracy: 0.8684
Epoch [6/10], Loss: 0.0003, Training Accuracy: 0.9986, Validation Accuracy: 0.8741
Epoch [7/10], Loss: 0.0002, Training Accuracy: 0.9981, Validation Accuracy: 0.8684
Epoch [8/10], Loss: 0.0003, Training Accuracy: 0.9906, Validation Accuracy: 0.8532
Epoch [9/10], Loss: 0.0004, Training Accuracy: 0.9975, Validation Accuracy: 0.8722
Epoch [10/10], Loss: 0.0004, Training Accuracy: 0.9953, Validation Accuracy: 0.8561


That's it!